In [6]:
from io import StringIO
import base64

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
import ssl

from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

nltk.download('wordnet')

from nltk import sent_tokenize
from nltk import word_tokenize

from nltk.stem import WordNetLemmatizer 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import torch
from transformers import BertTokenizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
df=pd.read_csv(r'C:\Users\hp\Downloads\gpt3.5_output10.csv')
df

,Unnamed: 0,message_id,subject,content,Body,result
0,0,<29892258.1075841922617.JavaMail.evans@thyme>,Confirm Letters - Grant County,Ive got one more deal that Houston says they ...,Confirm Letters - Grant County Ive got one mor...,"4) 'Logistic Arrangements (meeting scheduling,..."
1,1,<19756660.1075841923477.JavaMail.evans@thyme>,Re: deal 534056,This has been changed. Evelyn // : AM K...,Re: deal 534056 This has been changed. Ev...,"4) 'Logistic Arrangements(meeting scheduling, ..."
2,2,<27167573.1075841933378.JavaMail.evans@thyme>,Re: EESI #487723,This deal is part of a sleeve - we bought Pow...,Re: EESI #487723 This deal is part of a sleeve...,"4) 'Logistic Arrangements(meeting scheduling, ..."
3,3,<3253157.1075841933241.JavaMail.evans@thyme>,Re: #488882,Ive changed the deal to be confirmed. Mike Dr...,Re: #488882 Ive changed the deal to be confirm...,"4) 'Logistic Arrangements(meeting scheduling, ..."
4,4,<17966722.1075841923616.JavaMail.evans@thyme>,Daily EES Sales 2/28,Sorry for the delay -,Daily EES Sales 2/28 Sorry for the delay -,2) 'Purely Personal'
...,...,...,...,...,...,...
5782,25465,<18205073.1075860882146.JavaMail.evans@thyme>,Sun Devil Project - Preliminary Scoping and Fa...,Gentlemen As we discussed in our meeting las...,Sun Devil Project - Preliminary Scoping and Fa...,"1) 'Company Business, Strategy, etc.'"
5783,25466,<12247261.1075860881922.JavaMail.evans@thyme>,RE: Project Sun Devil EIS,Kevin I am still working with the Arizona Cha...,RE: Project Sun Devil EIS Kevin I am still wor...,"1) 'Company Business, Strategy, etc.'"
5784,25467,<24834367.1075860884605.JavaMail.evans@thyme>,Sun Devil Expansion Options,The attached & MMcf/d cases have been revie...,Sun Devil Expansion Options The attached & M...,"1) 'Company Business, Strategy, etc.'"
5785,25468,<28153820.1075860884743.JavaMail.evans@thyme>,RE: Az Public Service update on Sun Devil,Kevin Howard is the best contact on item . ...,RE: Az Public Service update on Sun Devil Kev...,"1) 'Company Business, Strategy, etc.'"


In [9]:
df.drop(['Unnamed: 0','Body'],axis=1,inplace=True)
df

C:\Users\hp\AppData\Local\Temp\ipykernel_12052\120272746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Unnamed: 0','Body'],axis=1,inplace=True)


,message_id,subject,content,result
4,<17966722.1075841923616.JavaMail.evans@thyme>,Daily EES Sales 2/28,Sorry for the delay -,2) 'Purely Personal'
16,<8565814.1075841938373.JavaMail.evans@thyme>,Re: Hotdogs today,You rock Angela! Dont worry Ill be my usual q...,2) 'Purely Personal'
17,<15282457.1075841937727.JavaMail.evans@thyme>,Re: Enpower: Counterparty Name Changes,Wow! I bet youre sick of explaining that one ...,3) 'Personal but in professional context'
21,<47627.1075841938054.JavaMail.evans@thyme>,Re: natsource,I changed this deal by the way. Please let me...,2) 'Purely Personal'
23,<5058485.1075841927113.JavaMail.evans@thyme>,Wrong CPs,Trades with TransAlta go through their energy...,"1) 'Company Business, Strategy, etc.'"
...,...,...,...,...
5781,<21225754.1075860882284.JavaMail.evans@thyme>,RE: AZ maps,Thanks Kevin. Scott J. Patti Senior Project ...,"1) 'Company Business, Strategy, etc.'"
5782,<18205073.1075860882146.JavaMail.evans@thyme>,Sun Devil Project - Preliminary Scoping and Fa...,Gentlemen As we discussed in our meeting las...,"1) 'Company Business, Strategy, etc.'"
5783,<12247261.1075860881922.JavaMail.evans@thyme>,RE: Project Sun Devil EIS,Kevin I am still working with the Arizona Cha...,"1) 'Company Business, Strategy, etc.'"
5784,<24834367.1075860884605.JavaMail.evans@thyme>,Sun Devil Expansion Options,The attached & MMcf/d cases have been revie...,"1) 'Company Business, Strategy, etc.'"


In [13]:
'''
# RE for filtering raw mail
df['message_id']=df['RawMailContent'].str.extract('message_id'+r':\s(.*)\n')
df['subject']=df['RawMailContent'].str.extract('Subject'+r':\s(.*)\n')
df['content']=df['RawMailContent'].str.extract('X-FileName'+r':\s.*\n([\s\S]*)$')
df.drop('RawMailContent',axis=1,inplace=True)
df['content']=df['content'].replace({'\n':' ','\t':' '},regex=True)

df['content']=df['content'].replace({'"':'',"'":""},regex=True)
df['subject']=df['subject'].replace({'"':'',"'":""},regex=True)

df.loc[df['content'].str.contains('-----Original Message-----'),'content']
df.loc[df['content'].str.contains('-----Original Message-----'),'content']
df.head()
'''

'\n# RE for filtering raw mail\ndf[\'message_id\']=df[\'RawMailContent\'].str.extract(\'message_id\'+r\':\\s(.*)\n\')\ndf[\'subject\']=df[\'RawMailContent\'].str.extract(\'Subject\'+r\':\\s(.*)\n\')\ndf[\'content\']=df[\'RawMailContent\'].str.extract(\'X-FileName\'+r\':\\s.*\n([\\s\\S]*)$\')\ndf.drop(\'RawMailContent\',axis=1,inplace=True)\ndf[\'content\']=df[\'content\'].replace({\'\n\':\' \',\'\t\':\' \'},regex=True)\n\ndf[\'content\']=df[\'content\'].replace({\'"\':\'\',"\'":""},regex=True)\ndf[\'subject\']=df[\'subject\'].replace({\'"\':\'\',"\'":""},regex=True)\n\ndf.loc[df[\'content\'].str.contains(\'-----Original Message-----\'),\'content\']\ndf.loc[df[\'content\'].str.contains(\'-----Original Message-----\'),\'content\']\ndf.head()\n'

In [14]:
df=df[~df['content'].isna()]
# df=df[df['labeled']==True]
col=['message_id','subject','content','cat_1_level_2']
df=df.loc[:,col]
df

KeyError: "['cat_1_level_2'] not in index"

In [15]:
df['content']=df['content'].replace({'\n':' ','\t':' '},regex=True)
df['content']=df['content'].replace({',':'',},regex=True)
df['content']=df['content'].replace({';':'',},regex=True)

df['content']=df['content'].replace({'"':'',},regex=True)
df['subject']=df['subject'].replace({'"':''},regex=True)

df['content']=df['content'].replace({"'":""},regex=True)
df['subject']=df['subject'].replace({"'":""},regex=True)

df=df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]
df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]

df.loc[df['content'].str.contains('-----Original Message-----'),'content']=df.loc[df['content'].str.contains('-----Original Message-----'),'content'].str.extract(r'([\s\S]*?)'+r'-----Original Message-----',expand=False)
df.loc[df['content'].str.contains('----- Forwarded by'),'content']=df.loc[df['content'].str.contains('----- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'----- Forwarded by',expand=False)
df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content']=df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'---------------------- Forwarded by',expand=False)
df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content']=df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content'].str.extract(r'([\s\S]*?)'+r'-------------------------- Sent from my BlackBerry',expand=False)

# remove all the numbers
df['content']=df['content'].replace({r'\d+':''},regex=True)

# remove email ids
df['content']=df['content'].replace({r'\S*@\S*\s?':''},regex=True)

'''
# remove everything after To: From: Re: Subject: etc..
df['content']=df['content'].replace({r'To:.+':'',r'From:.+':'',r'Re:.+':'',r'Subject:.+':'',r'Date:.+':''},regex=True)
'''

# remove just To: From: Re: Subject: etc.. texts
df['content']=df['content'].replace({r'To:':'',r'From:':'',r'Re:':'',r'Subject:':'',r'Date:':''},regex=True)




df

,message_id,subject,content,result
4,<17966722.1075841923616.JavaMail.evans@thyme>,Daily EES Sales 2/28,Sorry for the delay -,2) 'Purely Personal'
16,<8565814.1075841938373.JavaMail.evans@thyme>,Re: Hotdogs today,You rock Angela! Dont worry Ill be my usual q...,2) 'Purely Personal'
17,<15282457.1075841937727.JavaMail.evans@thyme>,Re: Enpower: Counterparty Name Changes,Wow! I bet youre sick of explaining that one ...,3) 'Personal but in professional context'
21,<47627.1075841938054.JavaMail.evans@thyme>,Re: natsource,I changed this deal by the way. Please let me...,2) 'Purely Personal'
23,<5058485.1075841927113.JavaMail.evans@thyme>,Wrong CPs,Trades with TransAlta go through their energy...,"1) 'Company Business, Strategy, etc.'"
...,...,...,...,...
5781,<21225754.1075860882284.JavaMail.evans@thyme>,RE: AZ maps,Thanks Kevin. Scott J. Patti Senior Project ...,"1) 'Company Business, Strategy, etc.'"
5782,<18205073.1075860882146.JavaMail.evans@thyme>,Sun Devil Project - Preliminary Scoping and Fa...,Gentlemen As we discussed in our meeting las...,"1) 'Company Business, Strategy, etc.'"
5783,<12247261.1075860881922.JavaMail.evans@thyme>,RE: Project Sun Devil EIS,Kevin I am still working with the Arizona Cha...,"1) 'Company Business, Strategy, etc.'"
5784,<24834367.1075860884605.JavaMail.evans@thyme>,Sun Devil Expansion Options,The attached & MMcf/d cases have been revie...,"1) 'Company Business, Strategy, etc.'"


In [16]:
'''
# removing duplicates
# df.rename(columns={'Message-ID': 'message_id'}, inplace=True)
df1=df1['Message-ID']
df1=list(df1)
df=df[~df['message_id'].isin(df1)]
df
'''

"\n# removing duplicates\n# df.rename(columns={'Message-ID': 'message_id'}, inplace=True)\ndf1=df1['Message-ID']\ndf1=list(df1)\ndf=df[~df['message_id'].isin(df1)]\ndf\n"

In [17]:
df['message_id']=df['message_id'].astype(str)
df['subject']=df['subject'].astype(str)
df['content']=df['content'].astype(str)

# to lower type
df['content']=df['content'].str.lower()
df['subject']=df['subject'].str.lower()

# remove punctuations
df['content'] = df['content'].str.replace('[^\w\s]','')
df['subject'] = df['subject'].str.replace('[^\w\s]','')

'''
# remove combination of words and digits
df['content'] = df['content'].str.replace('W*dw*','')
df['subject'] = df['subject'].str.replace('W*dw*','')
'''

# tokenization
df["content"] = df["content"].apply(lambda x: nltk.word_tokenize(x))
df["subject"] = df["subject"].apply(lambda x: nltk.word_tokenize(x))

'''
# porter stemming
ps = PorterStemmer()
df["content"] = df["content"].apply(lambda x: [ps.stem(i) for i in x])
df["subject"] = df["subject"].apply(lambda x: [ps.stem(i) for i in x])
'''
'''

# snowball stemming
ss = SnowballStemmer(language = "english")
df["content"] = df["content"].apply(lambda x: [ss.stem(i) for i in x])
df["subject"] = df["subject"].apply(lambda x: [ss.stem(i) for i in x])
'''

# lemmantization
lemmatizer = WordNetLemmatizer()
df["content"] = df["content"].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
df["subject"] = df["subject"].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

# join text back
def join_words(text):
    return " ".join(text)
df["subject"] = df["subject"].apply(lambda x: join_words(x))
df["content"] = df["content"].apply(lambda x: join_words(x))

df.to_csv('labelled_enron_10.csv')

C:\Users\hp\AppData\Local\Temp\ipykernel_12052\3943211426.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]','')
C:\Users\hp\AppData\Local\Temp\ipykernel_12052\3943211426.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['subject'] = df['subject'].str.replace('[^\w\s]','')


In [33]:
df

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,re confidential employee informationlenhart,i also need to know the base salary of jay rei...,1.0
1,<21041312.1075855725847.JavaMail.evans@thyme>,re personal and confidential compensation info...,thanks for the information it would be helpful...,3.0
5,<21261996.1075858638025.JavaMail.evans@thyme>,re western wholesale activity gas power conf c...,can you guy coordinate to make sure someone li...,4.0
6,<20399547.1075857614321.JavaMail.evans@thyme>,re confidential employee informationdutch quigley,thx jeanie slone pm john cc confidential emplo...,5.0
7,<860767.1075849626951.JavaMail.evans@thyme>,re hp confidential internal document,this is an excellent update thanks for putting...,1.0
...,...,...,...,...
1697,<18361957.1075861368310.JavaMail.evans@thyme>,confidential due diligence info g,there is a lot of confidential information her...,1.0
1698,<29049822.1075842045435.JavaMail.evans@thyme>,california dim bulb,you may have seen this when it first came out ...,1.0
1699,<205897.1075861997314.JavaMail.evans@thyme>,re confidential ee info,mark can you email me or fax me any written do...,5.0
1700,<23743848.1075863311776.JavaMail.evans@thyme>,confidential information and security trading,williams william enron wholesale service offic...,1.0


In [ ]:
df_content=df['content']
df_message_id=df['message_id']
df_subject=df['subject']
df_label=df['cat_1_level_2'].astype('int')

In [ ]:
df.to_csv('BERT_data.csv', index=False)

In [ ]:
# character length
df['content'].str.len().hist()

In [ ]:
# total word length
def sentence_len(sentence):
    return len(sentence.split())

df['content'].apply(lambda x: sentence_len(x)).hist()

In [ ]:
# average word length
df['content'].str.split().apply(lambda x : [len(i) for i in x]).map(lambda x: np.mean(x)).hist()

In [ ]:
# word count
new= df['content'].str.split()
new=df['content'].values.tolist()
corpus=[word for i in new for word in i.split()]
wo_li={}
for wo in corpus: 
    wo_li[wo] = wo_li.get(wo, 0) + 1    

In [ ]:
wo_li = dict(sorted(wo_li.items(), key=lambda item: item[1], reverse=True))
wo_li

pos_tags = pos_tag(wo_li.keys())
pos_tags

In [ ]:
pos_li={}
for x,pos in pos_tags: 
    pos_li[pos] = pos_li.get(pos, 0) + 1 
pos_li = dict(sorted(pos_li.items(), key=lambda item: item[1], reverse=True))
plt.figure(figsize=(10, 6))
sns.barplot(x=list(pos_li.keys())[:15],y=list(pos_li.values())[:15],palette='viridis')
plt.title('POS Distribution',size=22)
plt.xlabel('Part of Speech')
plt.ylabel('Count')

In [ ]:
pos_tags[1][1]

In [ ]:
nltk.download('stopwords')
stop=set(stopwords.words('english'))

In [ ]:
# stop words
stop_wo_li={}
for wo,i in wo_li.items():
    if wo in stop:
        stop_wo_li[wo]=i

plt.figure(figsize=(10, 6))
sns.barplot(x=list(stop_wo_li.keys())[:15],y=list(stop_wo_li.values())[:15],palette='viridis')
plt.title('Stop Words Distribution',size=22)
plt.xlabel('Stop Word')
plt.ylabel('Count')

In [ ]:
# not stop words
for st_wo in stop:
    if st_wo in wo_li:
        wo_li.pop(st_wo)
wo_li
plt.bar(list(wo_li.keys())[:5],list(wo_li.values())[:5])

In [ ]:
'''
# filtering out stop words
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)
df['content'] = df['content'].apply(remove_stop_words)
'''

In [ ]:
df["content"]

In [1]:
'''
# for removing names
import spacy
from spacy import displacy
from collections import Counter
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp('jim authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])
'''

In [ ]:
df.to_csv('BERT_data.csv', index=False)